# 1-D Linear Regression 

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline  
%config InlineBackend.figure_format='retina'
print ("TF version:[%s]."%(tf.__version__))

### Dataset

In [ ]:
W_ref,b_ref = 2+2*np.random.rand(),np.random.rand()
n = 1000 # number of data
x = np.random.rand(n,1)
y_ref = W_ref*x+b_ref
y = y_ref+0.5*np.random.randn(n,1)
print ("We have [%d] data."%(n))

### Plot training data

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(x,y,'k.',label='Noisy Data')
plt.plot(x,y_ref,'r.',label='Ref. Data')
plt.xlabel('x',fontsize=15); plt.ylabel('y',fontsize=15)
plt.legend(fontsize=15)
plt.show()

### Make a computational graph

In [ ]:
tf.reset_default_graph() # reset graph 
# Placeholders
ph_x = tf.placeholder(dtype=tf.float32,shape=(None,1),name='x')
ph_y = tf.placeholder(dtype=tf.float32,shape=(None,1),name='y')
# Forward graph
W = tf.Variable(np.sqrt(0.1)*np.random.randn(),dtype=tf.float32,name='W')
b = tf.Variable(0,dtype=tf.float32,name='b')
y_hat = W*ph_x + b
# Loss function
cost = tf.reduce_mean((y_hat-ph_y)**2) # MSE
optm = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
# Initializer
init = tf.global_variables_initializer()

### Run

In [ ]:
def gpu_sess():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    return sess
sess = gpu_sess() # open session

In [ ]:
sess.run(init) # Initialize variables
# Loop
max_iter,batch_size,print_every,plot_every = 5000,128,100,1000
for it in range(max_iter):
    # Optimize
    b_idx = np.random.permutation(n)[:batch_size]
    x_batch,y_batch = x[b_idx,:],y[b_idx,:]
    feeds = {ph_x:x_batch,ph_y:y_batch}
    cost_val,_ = sess.run([cost,optm],feed_dict=feeds)
    if ((it%print_every)==0) or (it==(max_iter-1)):
        [W_val,b_val] = sess.run([W,b])
        print ("it:[%d] cost_val:[%.3f] W:[%.2f] b:[%.2f] W:[%.2f] b:[%.2f]"%
               (it,cost_val,W_val,b_val,W_ref,b_ref))
    if ((it%plot_every)==0) or (it==(max_iter-1)):
        [W_val,b_val] = sess.run([W,b])
        y_val = W_val*x + b_val
        plt.figure(figsize=(8,4))
        plt.plot(x,y,'k.',label='Noisy Data');plt.plot(x,y_ref,'r.',label='Ref. Data')
        plt.plot(x,y_val,'b.',label='Fitted Data');plt.title("Iter:[%d]"%(it),fontsize=15)
        plt.xlabel('x',fontsize=15); plt.ylabel('y',fontsize=15);plt.legend(fontsize=15)
        plt.show()
print ("Done.")

### Now, use Class for more concise representation

In [ ]:
class LinearRegressionClass(object):
    def __init__(self,name='linear_regression',x_dim=1,y_dim=1):
        """
        Initialize Class
        """
        self.name = name
        self.x_dim = x_dim
        self.y_dim = y_dim
        # Build model and graph
        self.build_model()
        self.build_graph()
        print ("[%s] instantiated."%(self.name))
        
    def build_model(self):
        """
        Build forward model
        """
        # Placeholders
        self.ph_x = tf.placeholder(dtype=tf.float32,shape=(None,self.x_dim),name='x')
        self.ph_y = tf.placeholder(dtype=tf.float32,shape=(None,self.y_dim),name='y')
        # Forward graph
        self.W = tf.Variable(np.sqrt(0.1)*np.random.randn(),dtype=tf.float32,name='W')
        self.b = tf.Variable(0,dtype=tf.float32,name='b')
        self.y_hat = self.W*self.ph_x + self.b
        
    def build_graph(self):
        """
        Build computational graph
        """
        self.cost = tf.reduce_mean((self.y_hat-self.ph_y)**2) # MSE
        self.optm = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(self.cost)
        
    def update(self,sess,x_batch,y_batch):
        """
        Update with mini-batch
        """
        feeds = {self.ph_x:x_batch,self.ph_y:y_batch}
        cost_val,_ = sess.run([self.cost,self.optm],feed_dict=feeds)
        return cost_val
        
    def get(self,sess):
        """
        Get optimization variables 
        """
        [W_val,b_val] = sess.run([self.W,self.b])
        return [W_val,b_val]

def plot_progress(x,y,y_ref,y_val,it):
    plt.figure(figsize=(8,4))
    plt.plot(x,y,'k.',label='Noisy Data');plt.plot(x,y_ref,'r.',label='Ref. Data')
    plt.plot(x,y_val,'b.',label='Fitted Data');plt.title("Iter:[%d]"%(it),fontsize=15)
    plt.xlabel('x',fontsize=15); plt.ylabel('y',fontsize=15);plt.legend(fontsize=15)
    plt.show()

print ("Ready.")

### Instantiate Class

In [ ]:
tf.reset_default_graph()
sess = tf.Session() # open session
L = LinearRegressionClass()

In [ ]:
sess.run(tf.global_variables_initializer()) # Initialize variables
# Loop
max_iter,batch_size,print_every,plot_every = 5000,128,100,1000
for it in range(max_iter):
    # Optimize
    b_idx = np.random.permutation(n)[:batch_size]
    x_batch,y_batch = x[b_idx,:],y[b_idx,:]
    cost_val = L.update(sess=sess,x_batch=x_batch,y_batch=y_batch)
    if ((it%print_every)==0) or (it==(max_iter-1)):
        [W_val,b_val] = L.get(sess)
        print ("it:[%d] cost_val:[%.3f] W:[%.2f] b:[%.2f] W:[%.2f] b:[%.2f]"%
               (it,cost_val,W_val,b_val,W_ref,b_ref))
    if ((it%plot_every)==0) or (it==(max_iter-1)):
        [W_val,b_val] = L.get(sess)
        y_val = W_val*x + b_val
        plot_progress(x,y,y_ref,y_val,it)
print ("Done.")